## GCC118 - Programação Matemática
### Universidade Federal de Lavras
#### Luis Kennedy Gervásio Turola
#### Thaís Giovanna Lopes
#### Tobias Maugus Bueno Cougo

## Problema de Otimização: Cálculo de uma dieta

Uma nutricionista deseja formular uma dieta de emagrecimento, de forma a atender às necessidades mínimas de nutrientes de um indivíduo, respeitando limites calóricos e de custo, e controlando o excesso de alguns nutrientes específicos. Você dispõe de uma base de dados contendo 30 alimentos, cada um com informações de:

• preço por porção (R$),

• valor energético (kcal),

• proteína (g),

• gordura (g),

• carboidrato (g),

• cálcio (mg),

• ferro (mg),

• vitamina C (mg),

• sódio (mg).

### Regras da dieta
• A dieta diária deve conter no máximo 1600 kcal e no máximo R$ 20,00.

• Deve-se garantir um consumo mínimo de:

    – 60 g de proteína,

    – 130 g de carboidratos,

    – 1000 mg de cálcio,

    – 8 mg de ferro,

    – 75 mg de vitamina C.

• O consumo de gordura deve ser no máximo 50 g/dia.

• O consumo de sódio deve ser no máximo 2300 mg/dia.

### Objetivo

O objetivo não é minimizar o custo da dieta, mas sim minimizar a quantidade de nutrientes que ultrapassam os limites máximos permitidos.


## Modelo Matemático

### Declaração dos parâmetros

* $L$: conjunto de limites.  
* $A$: conjunto de alimentos.
* $N$: conjunto de nutrientes que devem ter os excessos considerados.
* $Min$: conjunto de itens com limitações de mínimo.
* $Max$: conjunto de itens com limitações de máximo.     
* $a_{mi} \in \mathbb{R}_+$: relação do atributo $m \in Min \cup Max \cup N$ no alimento $i \in A$.

### Variáveis de decisão

* $x_{i} \ge 0$: quantidade de porções do alimento $i \in A$.
* $e_{m} \ge 0$: excesso do nutriente $m \in N$ na dieta diária.

### Função objetivo
* Minimização dos excessos de nutrientes: $\min \sum_{j \in N} e_{j}$.

### Restrições
* $\sum_{m \in Min, i \in A} a_{mi}x_{i} \ge L_{m}, \forall m \in Min$.
* $\sum_{m \in N, i \in A} a_{mi}x_{i} - e_{m} \le L_{m}, \forall m \in N$.
* $\sum_{m \in Max, i \in A} a_{mi}x_{i} \le L_{m}, \forall m \in Max$.


## Resolução

In [2]:
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.9 MB/s eta 0:00:00


In [3]:
import json
from google.colab import files

# 1) Upload do arquivo JSON
print("Faça upload do arquivo JSON.")
uploaded = files.upload()
fname = next(iter(uploaded))
with open(fname, "r", encoding="utf-8") as f:
    data = json.load(f)

# 2) Extração de dados
params = data["params"]
alimentos = data["alimentos"]

# ids ordenados
ids = sorted(a["id"] for a in alimentos)

# monta dicionários por id (garantindo floats onde faz sentido)
def make_dict(key, default=0.0):
    return {a["id"]: float(a.get(key, default)) for a in alimentos}

preco    = make_dict("preco", 0.0)
kcal     = make_dict("kcal", 0.0)
proteina = make_dict("proteina", 0.0)
gordura  = make_dict("gordura", 0.0)
carbo    = make_dict("carboidrato", 0.0)
calcio   = make_dict("calcio", 0.0)
ferro    = make_dict("ferro", 0.0)
vitc     = make_dict("vitaminaC", 0.0)
sodio    = make_dict("sodio", 0.0)
nome     = {a["id"]: a.get("nome","") for a in alimentos}

# 3) Definição dos conjuntos (mapeando o nome do 'atributo' usado no alimento -> chave em parâmetros)
excessos_limitados = {
    "gordura": "fat_max",
    "sodio": "sod_max"
}

max_items = {
    "kcal": "Kmax",
    "preco": "Cmax",
}

min_items = {
    "proteina": "min_prot",
    "carboidrato": "min_carb",
    "calcio": "min_cal",
    "ferro": "min_fe",
    "vitaminaC": "min_vitc"
}

# 4) Mapeamento de atributo string para dicionário de dados usado acima
attr_map = {
    "preco": preco,
    "kcal": kcal,
    "proteina": proteina,
    "gordura": gordura,
    "carboidrato": carbo,
    "calcio": calcio,
    "ferro": ferro,
    "vitaminaC": vitc,
    "sodio": sodio
}

# 5) Criação do modelo
modelo = pulp.LpProblem("calculo_dieta", pulp.LpMinimize)

# 6) Criação das variáveis
# variáveis x_i (porções)
x = pulp.LpVariable.dicts("x", ids, lowBound=0, cat=pulp.LpInteger)

# cria as variáveis de excesso (para cada item em excessos_limitados)
excess_vars = {}
for attr, param_name in excessos_limitados.items():
    vname = f"excess_{attr}"
    excess_vars[attr] = pulp.LpVariable(vname, lowBound=0, cat=pulp.LpInteger)

# 6) Função objetivo: soma de excessos
# obj_terms = []
# for attr, var in excess_vars.items():
#     obj_terms.append(var)

modelo += pulp.lpSum(var for attr, var in excess_vars.items())

# 7) Definição das restrições dinamicamente

# excessos limitados: sum(attr * x) - excess_attr <= limite_max
for attr, param_name in excessos_limitados.items():
    limite = float(params[param_name])
    # pega o dicionário correspondente ao atributo
    d = attr_map[attr]
    modelo += (pulp.lpSum(d[i]*x[i] for i in ids) - excess_vars[attr] <= limite)

# máximos: sum(attr * x) <= limite_max
for attr, param_name in max_items.items():
    limite = float(params[param_name])
    # pega o dicionário correspondente ao atributo
    d = attr_map[attr]
    modelo += (pulp.lpSum(d[i]*x[i] for i in ids) <= limite)

# mínimos: sum(attr * x) >= limite_min
for attr, param_name in min_items.items():
    limite = float(params[param_name])
    d = attr_map[attr]
    modelo += (pulp.lpSum(d[i]*x[i] for i in ids) >= limite)

# 8) Resolve
resultado = modelo.solve()

# 9) Imprime solução
print("Status:", pulp.LpStatus[resultado])
print("Objective (soma ponderada de excessos):", modelo.objective.value())

tol = 1e-6
print("\nAlimentos usados:")
for i in ids:
    q = x[i].varValue or 0.0
    if q > tol:
      print(f" Id={i:2d} | {nome[i]:20s} | Porções = {q} | Preço por porção = R$ {preco[i]:.2f}")

# 10) Totais e excessos
print("\nNutrientes e valores totais:")
totals = {}
for attr in attr_map:
    totals[attr] = sum(attr_map[attr][i] * (x[i].varValue or 0.0) for i in ids)
    print(f"  {attr:12s}: {totals[attr]:10.2f}")

print("\nExcessos dos nutrientes:")
for attr, var in excess_vars.items():
    print(f"  {attr:12s} = {var.varValue}")

# 11) Custo total
total_cost = sum(preco[i] * (x[i].varValue or 0.0) for i in ids)
print(f"\nCusto total: R$ {total_cost:.2f}")

Faça upload do arquivo JSON.


Saving dieta.json to dieta (1).json
Status: Optimal
Objective (soma ponderada de excessos): 0.0

Alimentos usados:
 Id= 3 | Pão francês          | Porções = 3.0 | Preço por porção = R$ 0.40
 Id= 6 | Iogurte natural      | Porções = 3.0 | Preço por porção = R$ 1.20
 Id=10 | Sardinha enlatada    | Porções = 1.0 | Preço por porção = R$ 1.80
 Id=14 | Alface               | Porções = 4.0 | Preço por porção = R$ 0.40
 Id=19 | Laranja              | Porções = 1.0 | Preço por porção = R$ 0.60
 Id=25 | Lentilha             | Porções = 1.0 | Preço por porção = R$ 0.90

Nutrientes e valores totais:
  preco       :       9.70
  kcal        :    1160.00
  proteina    :      67.00
  gordura     :      25.90
  carboidrato :     160.00
  calcio      :    1030.00
  ferro       :      10.10
  vitaminaC   :      87.00
  sodio       :      27.00

Excessos dos nutrientes:
  gordura      = 0.0
  sodio        = 0.0

Custo total: R$ 9.70


In [4]:
modelo

calculo_dieta:
MINIMIZE
1*excess_gordura + 1*excess_sodio + 0.0
SUBJECT TO
_C1: - excess_gordura + x_1 + 10 x_10 + 0.1 x_11 + 1.5 x_12 + 0.2 x_13
 + 0.1 x_14 + 0.2 x_15 + 0.5 x_16 + 0.3 x_17 + 0.2 x_18 + 0.1 x_19 + 0.5 x_2
 + 0.3 x_20 + 15 x_21 + 18 x_22 + 16 x_23 + 0.5 x_24 + 0.4 x_25 + 3 x_26
 + 0.5 x_27 + 10 x_28 + 9 x_29 + 2 x_3 + 3 x_30 + 3.5 x_4 + 5 x_5 + 3 x_6
 + 7 x_7 + 15 x_8 + 2.5 x_9 <= 50

_C2: - excess_sodio + 10 x_11 + 15 x_13 + 5 x_14 + 6 x_15 + 60 x_16 + 10 x_17
 + 8 x_18 + x_2 + 10 x_21 + 2 x_22 + x_25 + 80 x_29 + 30 x_30 + 2 x_6 <= 2300

_C3: 130 x_1 + 180 x_10 + 100 x_11 + 200 x_12 + 25 x_13 + 15 x_14 + 40 x_15
 + 50 x_16 + 90 x_17 + 80 x_18 + 70 x_19 + 120 x_2 + 85 x_20 + 160 x_21
 + 200 x_22 + 190 x_23 + 110 x_24 + 115 x_25 + 150 x_26 + 100 x_27 + 90 x_28
 + 80 x_29 + 150 x_3 + 120 x_30 + 165 x_4 + 70 x_5 + 95 x_6 + 110 x_7
 + 250 x_8 + 120 x_9 <= 1600

_C4: 0.5 x_1 + 1.8 x_10 + 0.8 x_11 + 0.9 x_12 + 0.5 x_13 + 0.4 x_14 + 0.6 x_15
 + 0.8 x_16 + 0.5 x_17 + 0.7 x_18 